## 1st Step. Create a Notebook for the Assigment

# My Peer-Graded Assigment: Segmenting and Clustering Neighborhoods in Toronto

First Part is to install the proper libraries

In [1]:
#Installin Beautiful Soup
import sys
!{sys.executable} -m pip install bs4

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import os
import codecs
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


## 2nd Step. Get the information form Wikipedia

Second Part of the assigment is to create the dataframe in pandas of the neighborhood of Toronto

In [3]:
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
header = {
    'User-Agent': 'Mozilla/5.0'
} # Needed to prevent 403 error on Wikipedia
page = requests.get(wiki, headers=header)
soup = BeautifulSoup(page.content)

tables = soup.findAll("table", {"class": "wikitable"})

# show tables
for table in tables:
    print("###############")
    print(table.text[:100])

for tn in range(len(tables)):
    table = tables[tn]

    # preinit list of lists
    rows = table.findAll("tr")
    row_lengths = [len(r.findAll(['th', 'td'])) for r in rows]
    ncols = max(row_lengths)
    nrows = len(rows)
    data = []
    for i in range(nrows):
        rowD = []
        for j in range(ncols):
            rowD.append('')
        data.append(rowD)

    # process html
    for i in range(len(rows)):
        row = rows[i]
        rowD = []
        cells = row.findAll(["td", "th"])
        for j in range(len(cells)):
            cell = cells[j]

            #lots of cells span cols and rows so lets deal with that
            cspan = int(cell.get('colspan', 1))
            rspan = int(cell.get('rowspan', 1))
            l = 0
            for k in range(rspan):
                # Shifts to the first empty cell of this row
                while data[i + k][j + l]:
                    l += 1
                for m in range(cspan):
                    data[i + k][j + l + m] += cell.text

        data.append(rowD)

    # write data out to tab seperated format
    page = os.path.split(wiki)[1]
    fname = 'output_{}_t{}.csv'.format(page, tn)
    f = codecs.open(fname, 'w')
    for i in range(nrows):
        rowStr = '\t'.join(data[i])
        rowStr = rowStr.replace('\n', '')
        print(rowStr)
        f.write(rowStr + '\n')
    f.close()

###############


Postcode
Borough
Neighbourhood


M1A
Not assigned
Not assigned


M2A
Not assigned
Not assigned


M
Postcode	Borough	Neighbourhood
M1A	Not assigned	Not assigned
M2A	Not assigned	Not assigned
M3A	North York	Parkwoods
M4A	North York	Victoria Village
M5A	Downtown Toronto	Harbourfront
M5A	Downtown Toronto	Regent Park
M6A	North York	Lawrence Heights
M6A	North York	Lawrence Manor
M7A	Queen's Park	Not assigned
M8A	Not assigned	Not assigned
M9A	Etobicoke	Islington Avenue
M1B	Scarborough	Rouge
M1B	Scarborough	Malvern
M2B	Not assigned	Not assigned
M3B	North York	Don Mills North
M4B	East York	Woodbine Gardens
M4B	East York	Parkview Hill
M5B	Downtown Toronto	Ryerson
M5B	Downtown Toronto	Garden District
M6B	North York	Glencairn
M7B	Not assigned	Not assigned
M8B	Not assigned	Not assigned
M9B	Etobicoke	Cloverdale
M9B	Etobicoke	Islington
M9B	Etobicoke	Martin Grove
M9B	Etobicoke	Princess Gardens
M9B	Etobicoke	West Deane Park
M1C	Scarborough	Highland Creek
M1C	Scarborough	Rouge Hill
M1C

## 3rd Step. Create the information in Pandas Dataframe

### *First we put the info in a data frame and clean up the data as requested*

In [4]:
df = pd.read_csv('output_List_of_postal_codes_of_Canada:_M_t0.csv', sep='\t')
print(df.head())
print(df.shape)

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
(288, 3)


 ### *Ignore rows with Not assigned*

In [5]:
# Show the first 15 rows of the original dataframe and the shape to compare
print("Given Dataframe :\n", df.head(15))  
print("Shape of Given Dataframe", df.shape)

# Filter rows that does not content "Not assigned"
options = ['Not assigned']
df_tor = df.loc[~df['Borough'].isin(options)] 

# Show the first 15 rows of the resultant dataframe and shape
print('\nResult Dataframe :\n', df_tor.head(15)) 
print("Shape of Result Dataframe", df_tor.shape)

Given Dataframe :
    Postcode           Borough     Neighbourhood
0       M1A      Not assigned      Not assigned
1       M2A      Not assigned      Not assigned
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  Downtown Toronto      Harbourfront
5       M5A  Downtown Toronto       Regent Park
6       M6A        North York  Lawrence Heights
7       M6A        North York    Lawrence Manor
8       M7A      Queen's Park      Not assigned
9       M8A      Not assigned      Not assigned
10      M9A         Etobicoke  Islington Avenue
11      M1B       Scarborough             Rouge
12      M1B       Scarborough           Malvern
13      M2B      Not assigned      Not assigned
14      M3B        North York   Don Mills North
Shape of Given Dataframe (288, 3)

Result Dataframe :
    Postcode           Borough     Neighbourhood
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  D

 ### *Combine Rows with same Postcode adding Neighbourhood names with commas*

In [6]:
#Combine duplicated rows by groupby
df_tor1 = (df_tor.groupby(['Postcode','Borough'])['Neighbourhood']
       .apply(lambda x: ','.join(set(x.dropna())))
       .reset_index())

# Show the first 15 rows of the dataframe and shap
print('Dataframe :\n', df_tor1.head(15)) 
print("Shape of Dataframe", df_tor1.shape)

Dataframe :
    Postcode      Borough                                      Neighbourhood
0       M1B  Scarborough                                      Malvern,Rouge
1       M1C  Scarborough               Highland Creek,Rouge Hill,Port Union
2       M1E  Scarborough                    Morningside,Guildwood,West Hill
3       M1G  Scarborough                                             Woburn
4       M1H  Scarborough                                          Cedarbrae
5       M1J  Scarborough                                Scarborough Village
6       M1K  Scarborough          Ionview,Kennedy Park,East Birchmount Park
7       M1L  Scarborough                      Golden Mile,Clairlea,Oakridge
8       M1M  Scarborough      Cliffside,Scarborough Village West,Cliffcrest
9       M1N  Scarborough                         Cliffside West,Birch Cliff
10      M1P  Scarborough  Scarborough Town Centre,Wexford Heights,Dorset...
11      M1R  Scarborough                                   Wexford,Maryvale

 ### *Assign Neighborhood name in the Not Assigned Cells*

In [7]:
df_tor1.loc[df_tor1['Neighbourhood']=="Not assigned",'Neighbourhood']=df_tor1.loc[df_tor1['Neighbourhood']=="Not assigned",'Borough']
print('Dataframe :\n', df_tor1.tail(20)) 
print("Shape of Dataframe", df_tor1.shape)

Dataframe :
     Postcode       Borough                                      Neighbourhood
83       M6R  West Toronto                              Parkdale,Roncesvalles
84       M6S  West Toronto                                  Swansea,Runnymede
85       M7A  Queen's Park                                       Queen's Park
86       M7R   Mississauga              Canada Post Gateway Processing Centre
87       M7Y  East Toronto  Business Reply Mail Processing Centre 969 Eastern
88       M8V     Etobicoke         Mimico South,New Toronto,Humber Bay Shores
89       M8W     Etobicoke                              Alderwood,Long Branch
90       M8X     Etobicoke        The Kingsway,Montgomery Road,Old Mill North
91       M8Y     Etobicoke  Sunnylea,Kingsway Park South East,Mimico NE,Th...
92       M8Z     Etobicoke  Kingsway Park South West,The Queensway West,Mi...
93       M9A     Etobicoke                                   Islington Avenue
94       M9B     Etobicoke  Princess Gardens,West D

## Step 4. Put the file in my repostery Github

Output: "https://github.com/SuperIU/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb"

## Step 5. Assign coordenates for the postal codes in Toronto

### *First we import the coodinates form the site given in the Assigment*

In [8]:
# Get the file given in the Assigment
!wget -q -O 'Geopastical_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [9]:
# Get a dataframe with the postal codes and see if the list have the same rows of the Toronto DF
dfll = pd.read_csv('Geopastical_Coordinates.csv')
print('Postcode in Toronto :\n',dfll.head())
print("Shape of Postcode", dfll.shape)

Postcode in Toronto :
   Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
Shape of Postcode (103, 3)


### *We check both dataframes have the same rows. Now we orden in ascendent order both df and get the new columns*

In [10]:
# Order both df in ascending order
df_tor1.sort_values('Postcode')
dfll.sort_values('Postal Code')
#Check both dataframes heads and tails
print('First Rows of Toronto Post codes :\n',df_tor1.head())
print('Fists Rows of Coordinates Toronto :\n',dfll.head())
print('Last Rows of Toronto Post codes :\n',df_tor1.tail())
print('Last Rows of Coordinates Toronto :\n',dfll.tail())

First Rows of Toronto Post codes :
   Postcode      Borough                         Neighbourhood
0      M1B  Scarborough                         Malvern,Rouge
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2      M1E  Scarborough       Morningside,Guildwood,West Hill
3      M1G  Scarborough                                Woburn
4      M1H  Scarborough                             Cedarbrae
Fists Rows of Coordinates Toronto :
   Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
Last Rows of Toronto Post codes :
     Postcode    Borough                                      Neighbourhood
98       M9N       York                                             Weston
99       M9P  Etobicoke                                          Westmount
100      M9R  Etobicoke  Richview Gardens,Kingsview Village,St. Phillip...
101 

In [11]:
# Add the latitude and longitude columns in df_tor1 with the values of dfll
df_tor1['Latitude']=dfll['Latitude']
df_tor1['Longitude']=dfll['Longitude']
# Check the new dataframe 
print('First Rows of Toronto Post codes :\n',df_tor1.head())
print("Shape of Dataframe", df_tor1.shape)

First Rows of Toronto Post codes :
   Postcode      Borough                         Neighbourhood   Latitude  \
0      M1B  Scarborough                         Malvern,Rouge  43.806686   
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union  43.784535   
2      M1E  Scarborough       Morningside,Guildwood,West Hill  43.763573   
3      M1G  Scarborough                                Woburn  43.770992   
4      M1H  Scarborough                             Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  
Shape of Dataframe (103, 5)


# Step 6. Create a notebook with the result

Output: https://github.com/SuperIU/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto%20Part2.ipynb

# Step 7. Make the same excercise as New York

### *First I decide to make the exercsise with the borough Toronto*

In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_tor1['Borough'].unique()),
        df_tor1.shape[0]))

The dataframe has 11 boroughs and 103 neighborhoods.


In [13]:
df_tor1.head(25)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848


In [14]:
# create map of Toronto using latitude and longitude values
latitude = 43.7001100
longitude = -79.4163000
map_torontoarea = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_tor1['Latitude'], df_tor1['Longitude'], df_tor1['Borough'], df_tor1['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torontoarea)  
    
map_torontoarea

### *We select the Borough of Toronto and map it*

In [15]:
toronto_data = df_tor1[df_tor1['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
print(toronto_data.head())
print(toronto_data.shape)

  Postcode           Borough               Neighbourhood   Latitude  Longitude
0      M4W  Downtown Toronto                    Rosedale  43.679563 -79.377529
1      M4X  Downtown Toronto  St. James Town,Cabbagetown  43.667967 -79.367675
2      M4Y  Downtown Toronto        Church and Wellesley  43.665860 -79.383160
3      M5A  Downtown Toronto    Harbourfront,Regent Park  43.654260 -79.360636
4      M5B  Downtown Toronto     Ryerson,Garden District  43.657162 -79.378937
(18, 5)


In [16]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### *Connect with API Foursquare and check for places*

In [17]:
CLIENT_ID = 'FWIOM0WVIDQ4K2KBOWRONKQNRG3L0QATEDCTNCAUJE0XXJ0N' # your Foursquare ID
CLIENT_SECRET = 'LVQQKVQ0PAGO5VLVKWKRHOPMOCSY0G1K315RSIHWIFLLMD1U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FWIOM0WVIDQ4K2KBOWRONKQNRG3L0QATEDCTNCAUJE0XXJ0N
CLIENT_SECRET:LVQQKVQ0PAGO5VLVKWKRHOPMOCSY0G1K315RSIHWIFLLMD1U


In [18]:
# Get the neighbourhood name
toronto_data.loc[0, 'Neighbourhood']

'Rosedale'

In [19]:
# Get the coordinates 
neighbourhood_latitude = toronto_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = toronto_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


In [20]:
# Get de URL
LIMIT = 150 
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=FWIOM0WVIDQ4K2KBOWRONKQNRG3L0QATEDCTNCAUJE0XXJ0N&client_secret=LVQQKVQ0PAGO5VLVKWKRHOPMOCSY0G1K315RSIHWIFLLMD1U&v=20180605&ll=43.6795626,-79.37752940000001&radius=500&limit=150'

In [21]:
results = requests.get(url).json()

In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### *Cheking the nearest venues*

In [23]:
#filter results
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Mooredale House,Building,43.678631,-79.380091
1,Rosedale Park,Playground,43.682328,-79.378934
2,Whitney Park,Park,43.682036,-79.373788
3,Alex Murray Parkette,Park,43.678300,-79.382773
4,Milkman's Lane,Trail,43.676352,-79.373842


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Rosedale
St. James Town,Cabbagetown
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,Richmond,King
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
University of Toronto,Harbord
Kensington Market,Grange Park,Chinatown
South Niagara,Bathurst Quay,King and Spadina,Railway Lands,CN Tower,Harbourfront West,Island airport
Stn A PO Boxes 25 The Esplanade
Underground city,First Canadian Place
Christie


In [27]:
# size of the dataframe
print(toronto_venues.shape)
toronto_venues.head()

(1283, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


In [28]:
# Venues in each neighbourhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,Richmond,King",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
Central Bay Street,81,81,81,81,81,81
Christie,16,16,16,16,16,16
Church and Wellesley,85,85,85,85,85,85
"Commerce Court,Victoria Hotel",100,100,100,100,100,100
"Design Exchange,Toronto Dominion Centre",100,100,100,100,100,100
"Harbourfront East,Toronto Islands,Union Station",100,100,100,100,100,100
"Harbourfront,Regent Park",47,47,47,47,47,47


In [29]:
# unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 206 uniques categories.


### *Analize the venues*

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [31]:
toronto_onehot.shape

(1283, 207)

In [32]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide,Richmond,King",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.00,0.00,0.0,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.0,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,...,0.00,0.00,0.0,0.00,0.012346,0.000000,0.000000,0.012346,0.000000,0.012346
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.000000,...,0.00,0.00,0.0,0.00,0.000000,0.011765,0.011765,0.000000,0.011765,0.011765
5,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.00,0.00,0.0,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
6,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.00,0.00,0.0,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
7,"Harbourfront East,Toronto Islands,Union Station",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.0,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
8,"Harbourfront,Regent Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,...,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Kensington Market,Grange Park,Chinatown",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.01,0.00,0.0,0.00,0.060000,0.000000,0.040000,0.010000,0.000000,0.000000


In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### *Get the top venues to attach in the dataframe*

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,Richmond,King",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Breakfast Spot,Hotel,Burger Joint,Gym,Asian Restaurant
1,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Steakhouse,Cheese Shop,Café,Farmers Market,Jazz Club
2,Central Bay Street,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Burger Joint,Café,Middle Eastern Restaurant,Salad Place,Indian Restaurant,Chinese Restaurant
3,Christie,Café,Grocery Store,Park,Nightclub,Restaurant,Italian Restaurant,Baby Store,Diner,Athletics & Sports,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Gastropub,Men's Store,Mediterranean Restaurant,Hotel,Gym


### *Clustering the Neighbourhoods in Toronto*

In [35]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 4, 1, 1, 1, 1, 1, 1], dtype=int32)

In [36]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [37]:
toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Playground,Trail,Building,Yoga Studio,Diner,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M4X,Downtown Toronto,"St. James Town,Cabbagetown",43.667967,-79.367675,1,Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Market,Bakery,Park,Butcher
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Gastropub,Men's Store,Mediterranean Restaurant,Hotel,Gym
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Restaurant,Mexican Restaurant,Shoe Store,Brewery
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Fast Food Restaurant,Ramen Restaurant,Bookstore,Restaurant,Pizza Place
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Italian Restaurant,Café,Restaurant,Hotel,Clothing Store,Beer Bar,Cosmetics Shop,Cocktail Bar,Breakfast Spot
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Steakhouse,Cheese Shop,Café,Farmers Market,Jazz Club
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Burger Joint,Café,Middle Eastern Restaurant,Salad Place,Indian Restaurant,Chinese Restaurant
8,M5H,Downtown Toronto,"Adelaide,Richmond,King",43.650571,-79.384568,1,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Breakfast Spot,Hotel,Burger Joint,Gym,Asian Restaurant
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,1,Coffee Shop,Hotel,Aquarium,Café,Italian Restaurant,Sporting Goods Shop,Brewery,Scenic Lookout,Bakery,Pizza Place


In [38]:
#Cheking if there are NaN clusters in neighbourhoods
toronto_merged=toronto_merged.dropna()
toronto_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Playground,Trail,Building,Yoga Studio,Diner,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M4X,Downtown Toronto,"St. James Town,Cabbagetown",43.667967,-79.367675,1,Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Market,Bakery,Park,Butcher
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Gastropub,Men's Store,Mediterranean Restaurant,Hotel,Gym
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Restaurant,Mexican Restaurant,Shoe Store,Brewery
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Fast Food Restaurant,Ramen Restaurant,Bookstore,Restaurant,Pizza Place
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Italian Restaurant,Café,Restaurant,Hotel,Clothing Store,Beer Bar,Cosmetics Shop,Cocktail Bar,Breakfast Spot
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Steakhouse,Cheese Shop,Café,Farmers Market,Jazz Club
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Burger Joint,Café,Middle Eastern Restaurant,Salad Place,Indian Restaurant,Chinese Restaurant
8,M5H,Downtown Toronto,"Adelaide,Richmond,King",43.650571,-79.384568,1,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Breakfast Spot,Hotel,Burger Joint,Gym,Asian Restaurant
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,1,Coffee Shop,Hotel,Aquarium,Café,Italian Restaurant,Sporting Goods Shop,Brewery,Scenic Lookout,Bakery,Pizza Place


# Step 8. Creating the Map and Cluster Information

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,0,Café,Bar,Restaurant,Japanese Restaurant,Bookstore,Bakery,Italian Restaurant,Beer Bar,Beer Store,Nightclub


#### Cluster 2

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1,Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Market,Bakery,Park,Butcher
2,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Gastropub,Men's Store,Mediterranean Restaurant,Hotel,Gym
3,Downtown Toronto,1,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Restaurant,Mexican Restaurant,Shoe Store,Brewery
4,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Fast Food Restaurant,Ramen Restaurant,Bookstore,Restaurant,Pizza Place
5,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Café,Restaurant,Hotel,Clothing Store,Beer Bar,Cosmetics Shop,Cocktail Bar,Breakfast Spot
6,Downtown Toronto,1,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Beer Bar,Steakhouse,Cheese Shop,Café,Farmers Market,Jazz Club
7,Downtown Toronto,1,Coffee Shop,Ice Cream Shop,Italian Restaurant,Sandwich Place,Burger Joint,Café,Middle Eastern Restaurant,Salad Place,Indian Restaurant,Chinese Restaurant
8,Downtown Toronto,1,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Breakfast Spot,Hotel,Burger Joint,Gym,Asian Restaurant
9,Downtown Toronto,1,Coffee Shop,Hotel,Aquarium,Café,Italian Restaurant,Sporting Goods Shop,Brewery,Scenic Lookout,Bakery,Pizza Place
10,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Gastropub,Deli / Bodega,Bar,Bakery,Gym


#### Cluster 3

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Park,Playground,Trail,Building,Yoga Studio,Diner,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 4

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Plane,Sculpture Garden,Boutique,Boat or Ferry


#### Cluster 5

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,4,Café,Grocery Store,Park,Nightclub,Restaurant,Italian Restaurant,Baby Store,Diner,Athletics & Sports,Coffee Shop


# Step 9. Finally put the file in the repostaory Github

My conclusion is that the clustering wasn't ideal because:

1. Maybe are to many clusters to analyze  
1. Maybe is needed another approach to identified k and it's location  
1. Maybe we need to get all the data from Toronto

*A further analysis will be made*